# Custom Chatbot Project

# 1. Choose a Dataset and Explain the Scenario

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task
### Dataset: 2023_fashion_trends.csv

* I will be using the datasets provided by Udacity in the instructions : "2023_fashion_trends.csv" - This file contains reports and quotes about fashion trends for 2023. Each row includes the source URL, article title, and text snippet.
* I will demonstrate the use of custom dataset with an LLM.
* It is known that  OpenAI model "gpt-3.5-turbo-instruct" has the knowledge only till year 2021.
* If we make use of fashion_trends dataset provided by Udacity under /data folder, this data is till 2023. So we would be able to demonstrate that this data can help to augment the knowledge of LLM "gpt-3.5-turbo-instruct" by making use of the technique learnt i.e RAG (Retrival Augmentation Generation)


# 2. Prepare the Dataset for the Custom Query Process
TODO: In the cells below, load your chosen dataset into a pandas dataframe with a column named "text". This column should contain all of your text data, separated into at least 20 rows.

In [177]:
import pandas as pd

In [178]:
df = pd.read_csv('data/2023_fashion_trends.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     82 non-null     object
 1   Trends  82 non-null     object
 2   Source  82 non-null     object
dtypes: object(3)
memory usage: 2.0+ KB


In [179]:
df.describe()

,URL,Trends,Source
count,82,82,82
unique,5,82,5
top,https://www.whowhatwear.com/spring-summer-2023...,2023 Fashion Trend: Red. Glossy red hues took ...,Spring/Summer 2023 Fashion Trends: 21 Expert-A...
freq,42,1,42


In [180]:
df.head()

,URL,Trends,Source
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...


In [181]:
print(df.iloc[0]["URL"])
print(df.iloc[0]["Trends"])
print(df.iloc[0]["Source"])

https://www.refinery29.com/en-us/fashion-trends-2023
2023 Fashion Trend: Red. Glossy red hues took over the Fall 2023 runways ranging from Sandy Liang and PatBo to Tory Burch and Wiederhoeft. Think: Juicy reds with vibrant orange undertones that would look just as good in head-to-toe looks (see: a pantsuit) as accent accessory pieces (shoes, handbags, jewelry).
7 Fashion Trends That Will Take Over 2023 — Shop Them Now


In [182]:
df["text"] = df["Trends"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   URL     82 non-null     object
 1   Trends  82 non-null     object
 2   Source  82 non-null     object
 3   text    82 non-null     object
dtypes: object(4)
memory usage: 2.7+ KB


In [183]:
df.head()

,URL,Trends,Source,text
0,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Red. Glossy red hues took ...,7 Fashion Trends That Will Take Over 2023 — Sh...,2023 Fashion Trend: Red. Glossy red hues took ...
1,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Cargo Pants. Utilitarian w...,7 Fashion Trends That Will Take Over 2023 — Sh...,2023 Fashion Trend: Cargo Pants. Utilitarian w...
2,https://www.refinery29.com/en-us/fashion-trend...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...",7 Fashion Trends That Will Take Over 2023 — Sh...,"2023 Fashion Trend: Sheer Clothing. ""Bare it a..."
3,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Denim Reimagined. From dou...,7 Fashion Trends That Will Take Over 2023 — Sh...,2023 Fashion Trend: Denim Reimagined. From dou...
4,https://www.refinery29.com/en-us/fashion-trend...,2023 Fashion Trend: Shine For The Daytime. The...,7 Fashion Trends That Will Take Over 2023 — Sh...,2023 Fashion Trend: Shine For The Daytime. The...


In [184]:
df = df.drop(["URL", "Trends", "Source"], axis=1)
df.head()

,text
0,2023 Fashion Trend: Red. Glossy red hues took ...
1,2023 Fashion Trend: Cargo Pants. Utilitarian w...
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it a..."
3,2023 Fashion Trend: Denim Reimagined. From dou...
4,2023 Fashion Trend: Shine For The Daytime. The...


# 3. Perform the Custom Query Process

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [185]:
import openai
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key=""

### a. Send the query to the LLM without using the custom fashion trends data

In [186]:
query1 = """
"Tell me about the current fashion trends in the market"

"""
initial_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=query1,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_answer)

Some current fashion trends in the market include:
1. Sustainable and eco-friendly fashion: There has been a growing trend towards ethical and sustainable fashion, with more and more consumers seeking out brands that use environmentally friendly materials and production methods.

2. Nostalgic fashion: Nostalgia is a major trend in fashion, with many designers bringing back styles and trends from past decades, such as 70s inspired prints, 80s silhouettes, and 90s grunge.

3. Bold colors and patterns: Bright and bold colors and patterns are making a comeback in the fashion world. From neon hues to graphic prints, these eye-catching styles are popular among fashion influencers and trendsetters.

4. Athleisure: The trend


In [187]:
query2 = """
"What do you think about the animal prints?"

"""
initial_answer2 = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=query2,
    max_tokens=150
)["choices"][0]["text"].strip()
print(initial_answer2)

I am an AI and do not have personal opinions, but many people enjoy using animal prints in fashion and decor for their bold and unique look. Some may find them stylish and trendy, while others may not be as fond of them. Ultimately, it is a matter of personal preference.


## b. Generate Embeddings 

In [188]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,2023 Fashion Trend: Red. Glossy red hues took ...,"[-0.020833317190408707, -0.022008126601576805,..."
1,2023 Fashion Trend: Cargo Pants. Utilitarian w...,"[-0.001784870750270784, -0.02892744168639183, ..."
2,"2023 Fashion Trend: Sheer Clothing. ""Bare it a...","[-0.01045029703527689, -0.01917460933327675, 0..."
3,2023 Fashion Trend: Denim Reimagined. From dou...,"[-0.01555274985730648, -0.005349182989448309, ..."
4,2023 Fashion Trend: Shine For The Daytime. The...,"[-0.004937837366014719, 0.0018045977922156453,..."
...,...,...
77,"If lime green isn't your vibe, rest assured th...","[-0.002775615779682994, -0.018237359821796417,..."
78,"""As someone who can clearly (not fondly) remem...","[-0.01475916150957346, -0.0064850859344005585,..."
79,"""Combine this design shift with the fact that ...","[-0.02079174853861332, -0.025052573531866074, ..."
80,Thought party season ended at the stroke of mi...,"[-0.01981187053024769, -0.022380324080586433, ..."


In [189]:
#df.to_csv("embeddings.csv")
#df[["text", "embeddings"]].to_csv("embeddings.csv")
df.to_csv("embeddings.csv")

In [190]:
#df = pd.read_csv('embeddings.csv', index_col=0)
#df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
#df.head()

## c. Function help to find the cosine distance of input query with the dataframes rows 

In [191]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

def get_rows_by_cosine_distances(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    
    # Add distances column to the dataframe, which contains the cosine distances from of the query against each of the "text" 
    # row in in the dataframe.
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [192]:
get_rows_by_cosine_distances("Tell me about the current fashion trends in the market.", df)

,text,embeddings,distances
63,"""Every season, there is a trend that speaks to...","[-0.010750052519142628, -0.018666550517082214,...",0.160975
31,"The Tailored Look. This spring, I’m looking to...","[-0.023686544969677925, 0.0065940930508077145,...",0.169192
69,"""Leather jackets are leading the nouveau grung...","[-0.010424140840768814, 0.004091559909284115, ...",0.169296
52,I won't waste your time by quoting perhaps the...,"[-0.026018904522061348, -0.0006309617892839015...",0.170293
72,"""Sheer fashion dominated the trends conversati...","[-0.012702646665275097, 0.0010329753858968616,...",0.170657
...,...,...,...
15,Romantic Femininity. Balletcore is here to sta...,"[-0.043208468705415726, -0.012155783362686634,...",0.224679
34,Sheer Layering. I never in my life thought I w...,"[-0.013608495704829693, -0.010083802975714207,...",0.226525
78,"""As someone who can clearly (not fondly) remem...","[-0.01475916150957346, -0.0064850859344005585,...",0.228125
47,"""Big bags have become a favourite among celebr...","[-0.01633322611451149, -0.003675993997603655, ...",0.228128


In [193]:
get_rows_sorted_by_relevance("what are your views about animal prints?", df)

,text,embeddings,distances
54,Not a tie-dye girl yet fed up to the back teet...,"[-0.038696520030498505, 0.00490457471460104, -...",0.185116
55,"""The movement could be traced back to Jonathan...","[-0.03094329871237278, -7.339801959460601e-05,...",0.196293
52,I won't waste your time by quoting perhaps the...,"[-0.026018904522061348, -0.0006309617892839015...",0.206244
69,"""Leather jackets are leading the nouveau grung...","[-0.010424140840768814, 0.004091559909284115, ...",0.211688
53,"For spring 2023, there was a more surrealist i...","[-0.034433700144290924, -0.00899236835539341, ...",0.217043
...,...,...,...
78,"""As someone who can clearly (not fondly) remem...","[-0.01475916150957346, -0.0064850859344005585,...",0.263279
37,The-Bigger-the-Better Bags. Gone are the days ...,"[-0.011324513703584671, -0.0036169933155179024...",0.264725
6,2023 Fashion Trend: Cobalt Blue. The strongest...,"[-0.0069767870008945465, -0.021418340504169464...",0.264808
5,2023 Fashion Trend: Maxi Skirts. In response t...,"[-0.025447335094213486, -0.01921859383583069, ...",0.265055


## d. Function for creating the Prompt Template !!

In [194]:
import tiktoken

def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_by_cosine_distances(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)


#### Prompt Templates

In [195]:
print(create_prompt("Tell me about the current fashion trends in the market", df, 400))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

"Every season, there is a trend that speaks to the inner maximalist in us. While autumn gave us lurid neons, biker boots and Y2K glamour, spring/summer 2023 presents something a little sweeter, and those with an aversion to all things 'pretty,' consider this your warning to look away now," says Remy Farrell, shopping editor at Who What Wear UK.

###

"Leather jackets are leading the nouveau grunge trend, with Masha Popova, Knwls and Gucci bringing strong seasonal updates," says Gramston. See? Easy!

###

"Sheer fashion dominated the trends conversations for S/S 23 as seen at Simone Rocha, Christopher Kane, Rick Owens and Molly Goddard. We loved Molly Goddard's romantic take on the trend, featuring her signature ruffle details on sheer orange midi dresses," says Wiggins.

###

I won't waste your time by quoting perhaps the most famous line in fashio

In [196]:
print(create_prompt("what are your views about Animal Prints?", df, 400))


Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

Not a tie-dye girl yet fed up to the back teeth with ditsy florals? Then a new wave of digitally manipulated, mind-bending blurry prints will provide you with some sweet patterned relief for the coming season—they're certainly my number one choice when it comes to adding a new, unique motif into my heavily printed repertoire, says Hannah Almassi, editor in chief at Who What Wear UK.

###

"The movement could be traced back to Jonathan Anderson's cult-adored blur-print tube dresses from Loewe last summer—the designer kick-started a revival for formfitting trompe l'oeil and cyber prints, and the look appears to be holding strong for S/S 23. But there is also another side to this creative coin; I've noticed an air of bohemia for smudgy surface patterns, with some designers combining lighter-weight, billowing fabrics alongside painterly ombrés to great

## e. Answers a Question

In [197]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=1000
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    
    prompt = create_prompt(question, df, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

## 4. Write Questions to Demonstrate Custom Performance

 TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic Completion model query as well as the answer from your custom query.

## Question 1

In [198]:
custom_answer = answer_question("Tell me about the current fashion trends in the market", df)
print(custom_answer)

Currently, there are several fashion trends dominating the market. These include:
- Maximalist fashion with bright neon colors, biker boots, and Y2K glamour
- Neo-grunge with leather jackets and strong seasonal updates from designers like Masha Popova, Knwls, and Gucci
- Sheer fashion, particularly in dresses and tops, as seen in collections from Simone Rocha, Christopher Kane, Rick Owens, and Molly Goddard
- Pared-back and minimalist fashion with a focus on tailored pieces and crisp white shirting, as seen in the runways of Khaite, The Row, and Peter Do
- Nudity in a subtle and chic way, as seen in the shows of Saint Laurent and Valentino and reflected in a 600% increase in searches for sheer tops during Fashion Month
- Elevated basics, with a '90s influence and chic silhouettes, as seen in collections from Peter Do and Roksanda
- Red hues dominating the Fall 2023 runways from designers like Sandy Liang, PatBo, Tory Burch, and Wiederhoeft
- Denim being reimagined, with designers like 

## Question 2

In [199]:
custom_answer2 = answer_question("what are your views about leather jackets", df)
print(custom_answer2)

Based on the context, it appears that leather jackets are a popular trend in fashion, with many designers and brands incorporating them into their collections. They are seen as both elegant and edgy, and are often styled with other moto-inspired or biker-style clothing. There are various styles and updates to the traditional leather jacket, making it a versatile and sought-after piece in wardrobes. Additionally, it is mentioned that leather jackets have been a trend in past fashion seasons and continue to be popular for the upcoming season.


## 4. Write Questions to Demonstrate Custom Performance

In [200]:
print(f"""
Tell me some of the current fashion trends.

Original Answer: {initial_answer}
\n\nCustom Answer:   {custom_answer}

---

what are your views about oversized bags?
Original Answer: {initial_answer2}
\n\nCustom Answer:   {custom_answer2}
""")


Tell me some of the current fashion trends.

Original Answer: Some current fashion trends in the market include:
1. Sustainable and eco-friendly fashion: There has been a growing trend towards ethical and sustainable fashion, with more and more consumers seeking out brands that use environmentally friendly materials and production methods.

2. Nostalgic fashion: Nostalgia is a major trend in fashion, with many designers bringing back styles and trends from past decades, such as 70s inspired prints, 80s silhouettes, and 90s grunge.

3. Bold colors and patterns: Bright and bold colors and patterns are making a comeback in the fashion world. From neon hues to graphic prints, these eye-catching styles are popular among fashion influencers and trendsetters.

4. Athleisure: The trend


Custom Answer:   Currently, there are several fashion trends dominating the market. These include:
- Maximalist fashion with bright neon colors, biker boots, and Y2K glamour
- Neo-grunge with leather jackets 